# The shortest path problem as a transhipment problem.

## Introduction to optimization and operations research.

Michel Bierlaire


In [ ]:

import numpy as np
from IPython.core.display_functions import display
from matplotlib import pyplot as plt
from networkx import DiGraph
from teaching_optimization.networks import draw_network
from teaching_optimization.simplex_tableau import SimplexAlgorithmTableau
from teaching_optimization.tableau import SimplexTableau


In this lab, you will model the **shortest path** task as a **transhipment** (network flow) problem
and solve it using the simplex algorithm in two phases. You will build the linear optimization
problem by defining the cost of each arc (objective coefficients), constructing the node–arc
incidence matrix (flow conservation), and setting the right‑hand side to source/sink supply.
After solving, you will interpret the basic solution as a path and visualize it on the network.
The goal is to connect graph intuition (paths, arcs, nodes) with the algebra of a linear
optimization problem so you can recognize shortest path as a special case of transhipment.

Consider the four cities represented by the graph below. The travel
time (in minutes) by car is shown next to each arc connecting two
cities.

We want to determine the fastest itinerary from Orbe to Sierre.

- Model the problem as a transhipment problem.
- Solve it using the simplex algorithm in two phases.

# Description of the problem

In [ ]:

positions = {
    'Orbe': (-3.5, 0.5),
    'Lausanne': (-2.5, -1.5),
    'Fribourg': (1.5, 0.5),
    'Sierre': (4.5, -3),
}

nodes = list(positions.keys())

arcs = [
    ('Orbe', 'Fribourg', 36),
    ('Orbe', 'Lausanne', 28),
    ('Lausanne', 'Fribourg', 50),
    ('Lausanne', 'Sierre', 71),
    ('Fribourg', 'Sierre', 83),
    ('Fribourg', 'Lausanne', 50),
]

G: DiGraph = DiGraph()
for node in nodes:
    G.add_node(node, pos=positions[node])
G.add_weighted_edges_from(arcs, weight='cost')
fig, ax = plt.subplots(figsize=(8, 6))
draw_network(the_network=G, attr_edge_labels='cost', ax=ax)
plt.show()


# Building the transhipment problem

## Coefficients of the variables

In [ ]:
objective = np.array([cost for _, _, cost in arcs])


In [ ]:
print(objective)


## Constraint matrix.

- each row corresponds to a node,
- each column corresponds to an arc.

The column corresponding to arc (i,j) must contain a 1 at row i and a -1 at row j.
All elements of the column are zero.

We build a mapping between the node name and the row number.

In [ ]:
node_to_index = {node: i for i, node in enumerate(nodes)}
display(node_to_index)


Initialize the matrix

In [ ]:
num_nodes = len(nodes)
num_arcs = len(arcs)
matrix = np.zeros((num_nodes, num_arcs))


Populate the matrix

In [ ]:
for arc_index, (start, end, weight) in enumerate(arcs):
    row_start = node_to_index[start]
    row_end = node_to_index[end]
    matrix[row_start, arc_index] = 1
    matrix[row_end, arc_index] = -1


Here is the matrix

In [ ]:
display(matrix)


## Right hand side

It contains zero, except at the origin, where it is 1, and at the destination, where it is -1.

In [ ]:
rhs = np.zeros(num_nodes)
rhs[node_to_index['Orbe']] = 1.0
rhs[node_to_index['Sierre']] = -1.0
display(rhs)


# Solving the problem

Initialization

In [ ]:
the_algorithm = SimplexAlgorithmTableau(
    objective=objective, constraint_matrix=matrix, right_hand_side=rhs
)


Solving the problem

In [ ]:
optimal_tableau: SimplexTableau = the_algorithm.solve()


Optimal solution

In [ ]:
print(optimal_tableau.feasible_basic_solution)


# Interpret the solution and plot the shortest path.

Use the optimal tableau to identify the arcs in the shortest path

In [ ]:
shortest_path_arcs = list()
for index, value in enumerate(optimal_tableau.feasible_basic_solution):
    if value == 1:
        print(arcs[index])
        shortest_path_arcs.append(arcs[index])


Plot the graph including only the arcs in the shortest path.

In [ ]:
shortest_path: DiGraph = DiGraph()
for node in nodes:
    shortest_path.add_node(node, pos=positions[node])
shortest_path.add_weighted_edges_from(shortest_path_arcs, weight='cost')
fig, ax = plt.subplots(figsize=(8, 6))
draw_network(the_network=shortest_path, attr_edge_labels='cost', ax=ax)
plt.show()


Optimal value

In [ ]:
print(optimal_tableau.value_objective_function)